In [ ]:
todo: find a way to handle legacy posts

In [1]:
#index setting required due to index setting not being in place
import pandas as pd
with pd.HDFStore("store.h5") as store:
    pdf = store['pdf'].set_index('id').sort_index()
    cdf = store['cdf'].set_index(['parent_id', 'position']).sort_index()
    rdf = store['rdf'].set_index('name').sort_index()


In [2]:
#correcting two bugs in the original scraper script, don't wish to run it for another dozen hours again
from praw.models import Redditor

#first issue is that some praw objects were stored instread of author strings, this replaces them without fetching them from reddit
s = pdf['author']
pdf['author'] = s.apply(lambda x: x.name if isinstance(x, Redditor) else x)

#second, redditor entries found in the comments were duplicated due to a missing line that would have added them to the set
#there isn't an easy way to fix this without starting over
#I'm just dropping all author entires without a created data (used for throwawy detection)
rdf = rdf.dropna(subset=['created_utc'])

In [6]:
cdf.sample(10)

,,author,body,controversiality,created_utc,distinguished,edited,gilded,score
parent_id,position,,,,,,,,
t3_b8z9ed,86,None,NYA- people can and do break up after 4 years ...,0,2019-04-03 17:33:56,None,NaT,0,15
t3_b59mn5,27,notyourcoloringbook,NTA,0,2019-03-26 01:41:48,None,NaT,0,2
t3_bf81s0,13,badiatorsweg,Yeah you’re right prescription drugs aren’t th...,0,2019-04-20 03:42:09,None,NaT,0,-7
t3_bzg67o,5,FredFredFredFredFres,INFO. Did you ever confront him about not visi...,0,2019-06-11 18:25:28,None,NaT,0,2
t3_b04x43,8,Heighwayqueen,Ywbta It sounds like something has changed for...,0,2019-03-12 07:36:32,None,NaT,0,1
t3_b2i8xj,25,DoctorDashDash,Why do people post stories in AITA when they'r...,0,2019-03-18 15:32:04,None,NaT,0,11
t3_bo5wag,5,None,NAH - Theres nothing wrong with being proud of...,0,2019-05-13 16:51:48,None,NaT,0,18
t3_b9e9b8,116,DikkAntlers,ESH but very good idea,0,2019-04-04 21:49:08,None,NaT,0,1
t3_bnlhzz,4,alaskahro,NTA. It is ultimately your living space and yo...,0,2019-05-12 05:20:08,None,NaT,0,2


In [ ]:
pdf['link_flair_text'].value_counts()

In [ ]:
#returns a list of a_inf (needs more info/uncertain), a_op (OP is the asshole), a_to (the other person is the asshole)
def flair_to_label(row):
    flair = row['link_flair_text']
    if flair in ['Not enough info', 'not enough info','too close to call','Too close to call']:
        row['label'] = -1
        #row['a_inf'] = True
    elif flair in ['No A-holes here','no a--holes here','no assholes here']:
        row['label'] = 0
        #row['a_inf'], row['a_op'], row['a_to'] = [False, False, False]
    elif flair in ['Not the A-hole','not the a-hole','not the asshole']:
        row['label'] = 1
        #row['a_inf'], row['a_op'], row['a_to'] = [False, False, True]
    elif flair in ['Asshole','asshole']:
        row['label'] = 2
        #row['a_inf'], row['a_op'], row['a_to'] = [False, True, False]
    elif flair in ['Everyone Sucks','everyone sucks']:
        row['label'] = 3
        #row['a_inf'], row['a_op'], row['a_to'] = [False, True, True]
    return row

print('Total before flair processing:' + str(len(pdf)))
df = pdf.apply(flair_to_label,axis=1).dropna(subset=['label'])
print('Total after: ' + str(len(df)))
print(df['label'].value_counts())
#print(df['a_op'].value_counts())
#print(df['a_to'].value_counts())
df.columns

In [ ]:
#https://stackoverflow.com/questions/761824/python-how-to-convert-markdown-formatted-text-to-text
from markdown import Markdown
from io import StringIO


def unmark_element(element, stream=None):
    if stream is None:
        stream = StringIO()
    if element.text:
        stream.write(element.text)
    for sub in element:
        unmark_element(sub, stream)
    if element.tail:
        stream.write(element.tail)
    return stream.getvalue()


# patching Markdown
Markdown.output_formats["plain"] = unmark_element
__md = Markdown(output_format="plain")
__md.stripTopLevelTags = False


def unmark(text):
    return __md.convert(text)

In [ ]:
len(" It is a record of the post as originally written, in case the post is deleted or edited.*  \n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AmItheAsshole) if you have any questions or concerns.*")

In [ ]:
from datetime import timedelta
import re

#aita = re.compile(r"\baita\b",re.IGNORECASE)
#wibta = re.compile(r"\bwibta\b",re.IGNORECASE)
#deferring these for export notebook

def genFeats(row):
    #detect edit, attempt to undo if possible due to correlation between edits & YTA
    #we wish to avoid introducing spurrious correlations that BERT may pick up on
    #see: https://arxiv.org/abs/1907.07355
    row['isEdited'] = 0
    if row['edited'] == row['edited']:
        row['isEdited'] = 1
        for itx, comment in cdf.loc["t3_"+row.name].query('distinguished == distinguished').iterrows():
            if comment.body[-316:-280] == "\n\n*This is a copy of the above post.":
                row['isEdited'] = 0
                row['selftext'] = comment.body[:-316]
            if comment.body[:58] == "^^^^AUTOMOD  ***The following is a copy of the above post.":
                row['isEdited'] = 0
                row['selftext'] = comment.body[175:-188]
            if comment.body[:49] == "^^^^AUTOMOD  ***This is a copy of the above post.":
                row['isEdited'] = 0
                row['selftext'] = comment.body[143:-188]
                
    row['selftext'] = unmark(re.sub(r"&#x200B;\n?","",row['selftext']))
    #row['title'] = row['title'].replace("AITA","Am I the asshole").replace("WIBTA","Would I be the asshole")
    #.encode('utf-8') #
            
#    if row['author'] is not None:
#        row['isDeleted'] = False
#        try:
#            author = rdf.loc[row['author']]
#            if author is not None:
#                row['isThrowaway'] = (row['created_utc'] - author['created_utc'] < timedelta(days=3))
#            else:
#                row['isDeleted'] = True
#        except KeyError:
#            row['isThrowaway'] = None
#    else:
#        row['isDeleted'] = True
#    return row

#I don't think the throwaway check is really possible without going back to the scraper. Should have done it there.

    row['sum'] = 0
    nta, nah, esh, yta, info = [False] * 5
    
    def procComment(comment):
        nonlocal nta, nah, esh, yta, info, row
        row['sum'] += comment.score
        
        #INFO, NAH, NTA, YTA, ESH
        mask = [re.search(r"\binfo\b",comment.body,re.IGNORECASE) is not None,
               re.search(r"\bNAH\b",comment.body) is not None,
               (re.search(r"\bNTA\b",comment.body) or re.search(r"\bWBNTA\b",comment.body) \
               or re.search(r"\bYWBNTA\b",comment.body) or re.search(r"\bWNTA\b",comment.body)) is not None,
               (re.search(r"\bYTA\b",comment.body) or re.search(r"\bYWBTA\b",comment.body)) is not None,
               re.search(r"\bESH\b",comment.body) is not None
               ]
#        or "not the asshole" in comment.body.lower()\
#        or "not an asshole" in comment.body.lower():
#        or "you both are assholes" in comment.body.lower()
#        or "you are the asshole" in comment.body.lower()\
#        or "you're being an asshole" in comment.body.lower() \
#        or "you'd be an asshole" in comment.body.lower() \
#        or "you would be an asshole" in comment.body.lower() \
#        or "you're the asshole" in comment.body.lower():
        #info is commonly in lower case, but for some reason the 3 letter acronyms do not tend to be lowered
        if mask[0]:
            if 't_info' in row:
                row['t_info'] = max(row['t_info'], comment.score)
            else:
                row['t_info'] = comment.score
                row['m_info'] = comment.score
        
        fields = ['nah','nta','yta','esh']
        
        for i, name in enumerate(fields, start=1):
            if mask[i]:
                if 't_' + name in row:
                    row['t_'+ name] += comment.score
                else:
                    row['t_'+ name] = comment.score
                    
                #if this is the only match, then maxout
                if sum(mask) == 1:
                    if 'm_' + name in row:
                        row['m_'+ name] = max(comment.score,row['m_'+ name])
                    else:
                        row['m_'+ name] = comment.score
                        
    cdf.loc["t3_"+row.name].query('score > 0 and distinguished != distinguished and body != \"[deleted]\"').apply(procComment,axis=1)
    
    #we can drop rows with no values in an numexpr query
    for c in ['t_info','t_yta','t_nta','t_esh','t_nah','m_info','m_yta','m_nta','m_esh','m_nah']:
        if not c in row:
            row[c] = 0
    
    return row

In [ ]:
df.sample(10000).apply(genFeats,axis=1).query('isEdited == 1')
#ddf['isThrowaway'].value_counts()
#from  pytorch_transformers.tokenization_bert import BertTokenizer
#from pathlib import Path
#tokenizer = BertTokenizer.from_pretrained(Path('../BERT/uncased_L-12_H-768_A-12/'), do_lower_case=True)
#with open('test.txt','w',encoding='utf-8') as f:
#    for i, row in ddf.iterrows():
#        print(tokenizer.tokenize(row['selftext']))
#        f.write(row['title'] )
#        f.write('\n')
#        f.write(row['selftext'])
#        f.write('\n--------------------------\n')

In [ ]:
ddf = df.apply(genFeats,axis=1).drop(columns=['link_flair_css_class','link_flair_text','author'])
for l in ['locked','over_18','spoiler']:
    ddf[l] = ddf[l].apply(lambda x: 1 if x else 0)
with pd.HDFStore("store.h5") as store:
    store['ddf'] = ddf

In [ ]:
ddf['label'].value_counts()